In [11]:
# Deep Thought is simply a scratch pad for all things DNN/ML/Deep Learning related.

In [12]:
import numpy as np

In [27]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        """Return the output of the network if "a" is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The "mini_batch" is a list of tuples "(x, y)", and "eta"
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The "training_data" is a list of tuples
        "(x, y)" representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If "test_data" is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)
                
    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

In [24]:
net = Network([2, 3, 1])

In [15]:
net.sizes

[2, 3, 1]

In [16]:
net.biases

[array([[ 0.76531451],
        [-1.14889533],
        [-1.65272844]]), array([[ 1.67277167]])]

In [17]:
net.weights

[array([[-0.08224332, -0.50147207],
        [ 1.19778739, -0.94637526],
        [ 0.277976  ,  1.72053737]]),
 array([[ 1.33330276,  0.11625019, -0.03157473]])]

In [18]:
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.randn.html

In [20]:
zip(net.sizes[:-1], net.sizes[1:])

[(2, 3), (3, 1)]

In [21]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [28]:
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [29]:
import cPickle

In [30]:
import gzip

In [43]:
f = gzip.open('mnist.pkl.gz', 'rb')

In [44]:
tr_d, va_d, te_d = cPickle.load(f)

In [48]:
f.close()

In [49]:
training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]

In [50]:
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e 

In [51]:
training_results = [vectorized_result(y) for y in tr_d[1]]

In [52]:
training_data = zip(training_inputs, training_results)

In [53]:
validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]

In [54]:
validation_data = zip(validation_inputs, va_d[1])

In [55]:
test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]

In [56]:
test_data = zip(test_inputs, te_d[1])

In [57]:
net = Network([784, 30, 10])

In [37]:
import random

In [39]:
training_data

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
 array([5, 0, 4, ..., 8, 4, 8]))

In [40]:
test_data

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
 array([7, 2, 1, ..., 4, 5, 6]))

In [58]:
 net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 9003 / 10000
Epoch 1: 9243 / 10000
Epoch 2: 9337 / 10000
Epoch 3: 9412 / 10000
Epoch 4: 9409 / 10000
Epoch 5: 9401 / 10000
Epoch 6: 9468 / 10000
Epoch 7: 9464 / 10000
Epoch 8: 9454 / 10000
Epoch 9: 9484 / 10000
Epoch 10: 9465 / 10000
Epoch 11: 9488 / 10000
Epoch 12: 9492 / 10000
Epoch 13: 9526 / 10000
Epoch 14: 9508 / 10000
Epoch 15: 9497 / 10000
Epoch 16: 9521 / 10000
Epoch 17: 9534 / 10000
Epoch 18: 9526 / 10000
Epoch 19: 9502 / 10000
Epoch 20: 9511 / 10000
Epoch 21: 9545 / 10000
Epoch 22: 9529 / 10000
Epoch 23: 9534 / 10000
Epoch 24: 9533 / 10000
Epoch 25: 9562 / 10000
Epoch 26: 9548 / 10000
Epoch 27: 9545 / 10000
Epoch 28: 9525 / 10000
Epoch 29: 9520 / 10000
